In [1]:
import sys 
sys.path.append(rf"/nfs/home/genovese/thesis-wildfire-genovese/src")
from importlib import reload
import utils
reload(utils)
from utils import *

Choosing variogram model on ordinary kriging...

In [ ]:
scores = pd.read_csv('/nfs/home/genovese/thesis-wildfire-genovese/outputs/grid_searches/ordinary_target_kriging.csv')
scores[scores['rank_test_score'] == 1]

Ordinary spherical model selected

---

Fine tuning on ordinary spherical kriging...

In [ ]:
scores = pd.read_csv('/nfs/home/genovese/thesis-wildfire-genovese/outputs/grid_searches/fine_tuning_target_kriging.csv')
scores[scores['rank_test_score'] == 1].shape

Since the scores are the same, I'll use the model with the lowest fitting time

In [ ]:
scores[scores['mean_fit_time'] == scores['mean_fit_time'].min()].params.iloc[0]

{'nlags': 8, 'variogram_parameters': {'psill': 1.5, 'range': 1.5, 'nugget': -1}, 'weight': False}

In [3]:
from pykrige.rk import Krige
from sklearn.metrics import mean_squared_error

In [4]:
positives = gpd.GeoDataFrame(
    pd.DataFrame(gpd.read_file('/nfs/home/genovese/thesis-wildfire-genovese/data/data_loader_for_kriging/kriging_weather_grid.geojson')),
    ).set_crs(epsg=3857, allow_override=True)
negatives = gpd.GeoDataFrame(pd.DataFrame(
    gpd.read_file('/nfs/home/genovese/thesis-wildfire-genovese/data/data_loader_for_kriging/kriging_negative_weather_grid.geojson')),
    crs='EPSG:3857')

p = pd.concat([positives[['geometry']], pd.Series([1]*positives.shape[0], name='label')],  axis=1)
n = pd.concat([negatives[['geometry']], pd.Series([0]*negatives.shape[0], name='label')],  axis=1)

tot = pd.concat([p, n])

# Create Kriging model (scikit-learn wrapper)

kriging_model = Krige(method='ordinary', variogram_model='spherical', variogram_parameters={'psill': 1.5, 'range': 1.5, 'nugget': -1}, nlags=8, weight=False)
kriging_model.fit(np.array(pd.concat([tot.geometry.x, tot.geometry.y], axis=1)), tot.label)

In [9]:
import pickle 
with open('/nfs/home/genovese/thesis-wildfire-genovese/src/trained_models/target_kriging.pkl', 'wb') as f:
    pickle.dump(kriging_model, f)

In [ ]:
ambiguous = gpd.read_file('/nfs/home/genovese/thesis-wildfire-genovese/data/gathering_geojson/ambiguous_fires_grid.geojson')

In [ ]:
y_pred = kriging_model.predict(np.array(pd.concat([ambiguous.geometry.x, ambiguous.geometry.y], axis=1)))
pred_df = pd.concat([ambiguous, pd.Series(y_pred)], axis=1)

In [ ]:
pred_df.rename(columns={0: 'label'}, inplace=True)

In [ ]:
def reverse_day(year, day):
    feb = 28
    if year in list(range(2000, 2025, 4)):
        feb += 1
            
    days_per_month = [31, feb, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]

    for index, month in enumerate(days_per_month):
        if (day - month) == 0:
            return (month, index+1)
        elif (day - month) > 0:
            day -= month
        else:
            return (day, index+1)

In [ ]:
pred_df['DD'] = pred_df.apply(lambda x: reverse_day(x['YYYY'], x['day'])[0], axis=1)
pred_df['MM'] = pred_df.apply(lambda x: reverse_day(x['YYYY'], x['day'])[1], axis=1)
pred_df.to_file('/nfs/home/genovese/thesis-wildfire-genovese/data/data_loader_for_kriging/kriging_ambiguous_weather_grid.geojson', index=False, driver='GeoJSON')

In [ ]:
non_fires = gpd.read_file('/nfs/home/genovese/thesis-wildfire-genovese/data/data_loader_for_kriging/kriging_ambiguous_weather_grid.geojson')

In [ ]:
from pandas._libs.tslibs.timestamps import Timestamp
non_fires['initialdate'] = non_fires.apply(lambda x: Timestamp(year = x.YYYY, month=x.MM, day=x.DD), axis=1)
non_fires['id'] = [-(idx+100000) for idx in non_fires.index]

In [ ]:
non_fires['polygon'] = non_fires.buffer(1000)
fires = non_fires.set_geometry(col='polygon')
fires.drop(columns='geometry', inplace=True)
fires.rename(columns={'polygon': 'geometry'}, inplace=True)
fires.to_file('/nfs/home/genovese/thesis-wildfire-genovese/src/OnFireForest/sentinel_negative_fires/ambiguous.geojson',driver='GeoJSON', index=False)

---